In [17]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

pd.set_option('display.max_rows', None)

In [10]:
df = pd.read_csv('~/sorted_data.txt', sep='\t')

In [3]:
model_file = 'saved/BPR-Nov-21-2021_11-51-58.pth'
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file)

26 Nov 17:17    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain-small
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.0005
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 40960000
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = expo

In [6]:
if test_data.is_sequential:
    uid_list = test_data.dataset.inter_feat['user_id']
    positive_item = test_data.dataset.inter_feat['item_id']
else:
    uid_list = test_data.uid_list
    positive_item = test_data.uid2positive_item[uid_list]
    positive_item = torch.cat(list(positive_item))
assert(len(positive_item) == len(uid_list))
print(len(uid_list))
topk_score, topk_iid_list = full_sort_topk(uid_list, model, test_data, k=10, device=config['device'])
topk_score = topk_score.cpu()
topk_iid_list = topk_iid_list.cpu()
correct_mask = (positive_item.view(-1, 1) == topk_iid_list).any(1)

8793


/home/sankuai/recbole/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)


In [7]:
correct_user = uid_list[correct_mask]
correct_user

tensor([   7,   11,   14,  ..., 9940, 9947, 9968])

In [9]:
dataset.id2token('user_id', correct_user[0])

'40321606'

In [13]:
test_inter = test_data.dataset.inter_feat
inter = test_inter[test_inter['user_id'] == correct_user[0]]

In [15]:
print(inter['user_id'], inter['item_id'])

tensor([7]) tensor([111632])


In [16]:
dataset.id2token('item_id', 111632)

'1863468494_POI_XIUYU'

In [18]:
df[df['user_id'] == 40321606]

,user_id,item_id,name,time,inter_type,user_loc
16388,40321606,698306932_DEAL_GROUP,悦秀·高级粤菜悦秀【午餐点心自助双人券】,2021-05-29 00:18:46,click,116.9723 36.6369
16389,40321606,696906623_DEAL_GROUP,MSKY摩登餐厅摩登双人餐，提供免费WiFi,2021-05-29 00:19:28,click,116.9723 36.6369
16390,40321606,192710368_POI_GROUP,茉小莉（恒隆广场店）,2021-05-29 00:31:39,click,116.9722 36.6371
16391,40321606,647030228_DEAL_GROUP,非尝锅氣畅销2人套餐，提供免费WiFi,2021-05-29 00:32:42,click,116.9722 36.6371
16392,40321606,163588996_POI_GROUP,大咀港式茶餐厅（和谐广场店）,2021-05-29 00:33:03,click,116.9723 36.6370
16393,40321606,2013221805_SPU_SHANGOU,【只愿博得妃子一笑】33荔枝花束,2021-06-01 08:16:07,click,116.9336 36.6301
16394,40321606,47840121_DEAL_GROUP,My-Forist花艺生活馆特价爆款手提花盒送朋友送闺蜜精致花篮,2021-06-01 10:20:45,click,116.9337 36.6300
16395,40321606,667759256_PRODUCT,甜宠旺旺礼盒水果鲜花礼盒鲜花私人定制鲜花礼物生日鲜花,2021-06-01 10:58:27,click,116.9337 36.6300
16396,40321606,2013221805_SPU_SHANGOU,【只愿博得妃子一笑】33荔枝花束,2021-06-01 11:02:50,click,116.9337 36.6300
16397,40321606,170660242_POI_WAIMAI,皇家龙虾（二七店）,2021-06-04 10:47:27,click,116.9883 36.6281
